In [1]:
%load_ext autoreload
%autoreload 2
%cd 'python/AggNet'
%ls

/home/hew/python/AggNet
APNet_CPAD2_results.csv  checkpoint/  figure/     README.md  utils/
APNet_results.csv        data/        install.sh  script/
APRNet_results.csv       dataset/     model/      temp/


In [2]:
import pandas as pd

from utils.file import write_fasta

==================== add root_path to sys.path ====================
root_path: /home/hew/python/AggNet


In [3]:
df = pd.read_excel('./data/CPAD2/aggregating peptides.xlsx')
df

,Entry,Peptide,Length,Classification,Uniprot Entry Name,Uniprot ID,Protein Name,Position,% Helix (PASTA 2.0),% Beta Strand (PASTA 2.0),...,Tango,% Disorder (PASTA 2.0),Best Energy Score (PASTA 2.0),Aggregate Orientation (PASTA 2.0),Mutant,Protein Sequence,Amyloid Structure,Reference,PMID,"Source (Waltz-DB,CPAD,AmyLoad,Waltz)"
0,P-0001,GYE,3,Amyloid,A4_HUMAN,P05067,Amyloid-beta precursor protein (APP),680-682,0.000000,0.000000,...,0.00,100.000000,10000.000000,Antiparallel,No,MLPGLALLLLAAWTARALEVPTDGNAGLLAEPQIAMFCGRLNMHMN...,No structures,Org Lett. 2008 Jul 3;10(13):2625-8.,18529009,CPAD
1,P-0002,KFFE,4,Amyloid,NaN,NaN,NaN,NaN,0.000000,0.000000,...,0.00,100.000000,-2.222498,Antiparallel,NaN,NaN,No structures,J Biol Chem. 2002 Nov 8;277(45):43243-6.,12215440,CPAD
2,P-0003,KVVE,4,Amyloid,NaN,NaN,NaN,NaN,0.000000,0.000000,...,0.00,100.000000,-2.838181,Antiparallel,NaN,NaN,No structures,J Biol Chem. 2002 Nov 8;277(45):43243-6.,12215440,CPAD
3,P-0004,NNQQ,4,Amyloid,ERF3_YEAST,P05453,Eukaryotic peptide chain release factor GTP-bi...,8-11,0.000000,0.000000,...,0.00,100.000000,1.802299,Antiparallel,NaN,MSDSNQGNNQQNYQQYSQNGNQQQGNNRYQGYQAYNAQAQPAGGYY...,No structures,Nature. 2007 May 24;447(7143):453-7.,17468747,CPAD
4,P-0005,VKSE,4,Non-amyloid,TAU_HUMAN,P10636,Microtubule-associated protein tau (PHF-tau),NaN,0.000000,0.000000,...,0.00,100.000000,0.586433,Antiparallel,NaN,NaN,No structures,Proc Natl Acad Sci U S A. 2000 May 9;97(10):51...,10805776,AmyLoad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026,P-2027,GQGGGTHNQWNKPSKPKTNMKHMAGAAAAGAVVGGLGGYMLGSAMS...,56,Amyloid,PRIO_SHA,NaN,NaN,NaN,44.642857,0.000000,...,15.38,41.071429,-2.172841,Parallel,NaN,NaN,No structures,Nature. 2007 May 24;447(7143):453-7.,17468747,CPAD
2027,P-2028,GQPHGGGWGQGGGTHSQWNKPSKPKTNMKHMAGAAAAGAVVGGLGG...,65,Amyloid,PRIO_HUMAN,P04156,Major prion protein (PrP),82-146,30.769231,7.692308,...,16.50,63.076923,-4.751561,Parallel,No,MANLGCWMLVLFVATWSDLGLCKKRPKPGGWNTGGSRYPGQGSPGG...,No structures,Biochemistry. 2006 May 30;45(21):6724-32.,16716083,CPAD
2028,P-2029,MATLEKLMKAFESLKSFQQQQQQQQQQQQQQQQQQQQQQQQQQQQQ...,71,Amyloid,NaN,NaN,NaN,NaN,77.464789,0.000000,...,0.00,100.000000,-1.123618,Antiparallel,NaN,NaN,No structures,Biophys J. 2011 May 18;100(10):2504-12.,21575585,CPAD
2029,P-2030,GKTKEGVLYVGSKTKEGVVHGVATVAEKTKEQVTNVGGAVVTGVTA...,79,Amyloid,SYUA_HUMAN,P37840,Alpha-synuclein,31-109,2.531646,48.101266,...,806.49,22.784810,-7.239327,Parallel,No,MDVFMKGLSKAKEGVVAAAEKTKQGVAEAAGKTKEGVLYVGSKTKE...,No structures,J Biol Chem. 2002 May 24;277(21):19213-9.,11893734,AmyLoad


In [4]:
df['Classification'].value_counts()

Classification
Non-amyloid    1055
Amyloid         716
amyloid         201
non-amyloid      59
Name: count, dtype: int64

In [5]:
sequences = df['Peptide'].values

text2label = {
    'Amyloid': 1,
    'amyloid': 1,
    'Non-amyloid': 0,
    'non-amyloid': 0
}
labels = df['Classification'].apply(lambda x: text2label[x]).values
entries = df['Entry'].values

is_amyloid = {
    1: 'amyloid',
    0: 'non-amyloid'
}
headers = [f'{entry}, {is_amyloid[label]}, label={label}' for entry, label in zip(entries, labels)]

In [6]:
path = './data/CPAD2/CPAD2.fasta'
write_fasta(path, sequences, headers)

In [7]:
CPAD2_df = df[['Peptide', 'Classification']].copy()
CPAD2_df['Classification'] = CPAD2_df['Classification'].apply(lambda x: text2label[x])
CPAD2_df.columns = ['peptide', 'label']
CPAD2_df

,peptide,label
0,GYE,1
1,KFFE,1
2,KVVE,1
3,NNQQ,1
4,VKSE,0
...,...,...
2026,GQGGGTHNQWNKPSKPKTNMKHMAGAAAAGAVVGGLGGYMLGSAMS...,1
2027,GQPHGGGWGQGGGTHSQWNKPSKPKTNMKHMAGAAAAGAVVGGLGG...,1
2028,MATLEKLMKAFESLKSFQQQQQQQQQQQQQQQQQQQQQQQQQQQQQ...,1
2029,GKTKEGVLYVGSKTKEGVVHGVATVAEKTKEQVTNVGGAVVTGVTA...,1


In [8]:
pred_df = pd.read_csv('./APNet_CPAD2_results.csv')
pred_df['pred'] = pred_df['probability'].apply(lambda x: 1 if x > 0.5 else 0)
pred_df = pred_df.drop(columns=['label'])
pred_df

,peptide,probability,pred
0,GYE,0.329107,0
1,KFFE,0.182013,0
2,KVVE,0.519902,1
3,NNQQ,0.222988,0
4,VKSE,0.134442,0
...,...,...,...
2026,GQGGGTHNQWNKPSKPKTNMKHMAGAAAAGAVVGGLGGYMLGSAMS...,0.075622,0
2027,GQPHGGGWGQGGGTHSQWNKPSKPKTNMKHMAGAAAAGAVVGGLGG...,0.055730,0
2028,MATLEKLMKAFESLKSFQQQQQQQQQQQQQQQQQQQQQQQQQQQQQ...,0.137469,0
2029,GKTKEGVLYVGSKTKEGVVHGVATVAEKTKEQVTNVGGAVVTGVTA...,0.203601,0


In [9]:
merge_df = pd.merge(CPAD2_df, pred_df, on='peptide', how='inner')
merge_df['length'] = merge_df['peptide'].apply(len)
merge_df

,peptide,label,probability,pred,length
0,GYE,1,0.329107,0,3
1,KFFE,1,0.182013,0,4
2,KVVE,1,0.519902,1,4
3,NNQQ,1,0.222988,0,4
4,VKSE,0,0.134442,0,4
...,...,...,...,...,...
2025,GQGGGTHNQWNKPSKPKTNMKHMAGAAAAGAVVGGLGGYMLGSAMS...,1,0.075622,0,56
2026,GQPHGGGWGQGGGTHSQWNKPSKPKTNMKHMAGAAAAGAVVGGLGG...,1,0.055730,0,65
2027,MATLEKLMKAFESLKSFQQQQQQQQQQQQQQQQQQQQQQQQQQQQQ...,1,0.137469,0,71
2028,GKTKEGVLYVGSKTKEGVVHGVATVAEKTKEQVTNVGGAVVTGVTA...,1,0.203601,0,79


In [10]:
from sklearn.metrics import roc_auc_score

In [11]:
temp_df = merge_df[merge_df['length'] == 5]
pred = temp_df['pred'].values
label = temp_df['label'].values
auc = roc_auc_score(label, pred)
auc

np.float64(0.7083333333333334)

In [12]:
temp_df = merge_df[merge_df['length'] == 6]
pred = temp_df['pred'].values
label = temp_df['label'].values
auc = roc_auc_score(label, pred)
auc

np.float64(0.8879045799316774)

In [13]:
temp_df = merge_df[merge_df['length'] == 7]
pred = temp_df['pred'].values
label = temp_df['label'].values
auc = roc_auc_score(label, pred)
auc

np.float64(0.6960132890365449)

In [14]:
temp_df = merge_df[merge_df['length'] == 8]
pred = temp_df['pred'].values
label = temp_df['label'].values
auc = roc_auc_score(label, pred)
auc

np.float64(0.7403846153846154)

In [15]:
temp_df = merge_df[merge_df['length'] == 9]
pred = temp_df['pred'].values
label = temp_df['label'].values
auc = roc_auc_score(label, pred)
auc

np.float64(0.7142857142857143)